In [ ]:
import cv2
import mediapipe as mp
import math

mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
mp_style = mp.solutions.drawing_styles

def dist(p1, p2):
    return math.sqrt((p1.x - p2.x)**2 + (p1.y - p2.y)**2)

# функция определения улыбки с интенсивностью и средней точкой рта
def smile_intensity(face):
    # ключевые точки
    left_corner = face[61]
    right_corner = face[291]
    top_lip = face[13]
    bottom_lip = face[14]

    # средняя точка рта
    mouth_center_y = (top_lip.y + bottom_lip.y) / 2
    middle_point = ((left_corner.x + right_corner.x)/2, (left_corner.y + right_corner.y)/2)

    # поднятие уголков рта относительно центра рта
    left_lift = mouth_center_y - left_corner.y
    right_lift = mouth_center_y - right_corner.y

    # ширина рта
    mouth_width = dist(left_corner, right_corner)

    # нормированная метрика улыбки
    smile_score = ((left_lift + right_lift)/2) / mouth_width
    smile_score = max(0.0, min(smile_score, 0.2))
    intensity = int(smile_score / 0.2 * 100)

    smiling = intensity > 10

    return smiling, middle_point, intensity

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cv2.namedWindow("Smile & Eyes Detection", cv2.WINDOW_NORMAL)

with mp_face_mesh.FaceMesh(
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as face_mesh:

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(rgb)

        smile = False
        intensity = 0
        mouth_middle = (0, 0)
        eyes_open = False

        if results.multi_face_landmarks:
            face = results.multi_face_landmarks[0].landmark

            # определяем улыбку
            smile, mouth_middle, intensity = smile_intensity(face)

            # проверка глаз (EAR)
            left_eye_EAR = dist(face[159], face[145])
            right_eye_EAR = dist(face[386], face[374])
            eyes_open = left_eye_EAR > 0.015 and right_eye_EAR > 0.015

            # рисуем сетку лица
            mp_drawing.draw_landmarks(
                image=frame,
                landmark_list=results.multi_face_landmarks[0],
                connections=mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_style.get_default_face_mesh_tesselation_style()
            )
            # рисуем контуры губ
            mp_drawing.draw_landmarks(
                image=frame,
                landmark_list=results.multi_face_landmarks[0],
                connections=mp_face_mesh.FACEMESH_LIPS,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_style.get_default_face_mesh_contours_style()
            )

        cv2.putText(frame, f"Smile: {smile} ({intensity}%)", (10, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0) if smile else (0, 0, 255), 2)

        cv2.putText(frame, f"Eyes:  {eyes_open}", (10, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0) if eyes_open else (0, 0, 255), 2)

        # рисуем среднюю точку рта
        cv2.circle(frame, (int(mouth_middle[0]*frame.shape[1]), int(mouth_middle[1]*frame.shape[0])), 5, (0,0,255), -1)

        cv2.imshow("Smile & Eyes Detection", frame)

        if cv2.waitKey(1) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import dlib
import numpy as np
from imutils import face_utils

predictor_path = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

# Функция для вычисления EAR (Eye Aspect Ratio)
def eye_aspect_ratio(eye):
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Функция для проверки улыбки
def is_smiling(landmarks):
    left_corner = landmarks[48]
    right_corner = landmarks[54]
    top_lip = landmarks[51]
    bottom_lip = landmarks[57]

    # средняя точка между уголками рта
    middle_point = ((left_corner + right_corner) / 2).astype(int)

    # ширина и высота рта
    mouth_width = np.linalg.norm(left_corner - right_corner)
    mouth_height = np.linalg.norm(top_lip - bottom_lip)

    # вертикальное смещение уголков рта относительно средней линии рта
    mouth_center_y = (top_lip[1] + bottom_lip[1]) / 2
    left_lift = mouth_center_y - left_corner[1]
    right_lift = mouth_center_y - right_corner[1]

    smile_metric = (left_lift + right_lift)/2 + (mouth_width / mouth_height)

    is_smile = smile_metric > 7.0

    return is_smile, middle_point

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        shape = predictor(gray, face)
        shape = face_utils.shape_to_np(shape)

        for (i, (x, y)) in enumerate(shape):
            cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)
            cv2.putText(frame, str(i), (x-10, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 0), 1)

        # Маска лица (convex hull)
        hull = cv2.convexHull(shape)
        cv2.drawContours(frame, [hull], -1, (255, 0, 0), 2)

        # Проверка глаз (левый и правый)
        left_eye = shape[42:48]
        right_eye = shape[36:42]
        left_ear = eye_aspect_ratio(left_eye)
        right_ear = eye_aspect_ratio(right_eye)
        ear = (left_ear + right_ear) / 2.0

        # Проверка улыбки
        smiling, mouth_middle = is_smiling(shape)

        eye_status = "True" if ear > 0.25 else "False"
        smile_status = "True" if smiling else "False"

        cv2.circle(frame, tuple(mouth_middle), 3, (0, 0, 255), -1)
        cv2.putText(frame, f"Eyes: {eye_status}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(frame, f"Smile: {smile_status}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        cv2.drawContours(frame, [cv2.convexHull(left_eye)], -1, (0, 255, 255), 1)
        cv2.drawContours(frame, [cv2.convexHull(right_eye)], -1, (0, 255, 255), 1)

    cv2.imshow("Face Analysis", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import mediapipe as mp

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

def dist(p1, p2):
    return math.sqrt((p1.x - p2.x)**2 + (p1.y - p2.y)**2)

cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5,
                  min_tracking_confidence=0.5) as pose:

    hands_stage = "down"
    clap_stage = "apart"

    hands_up_counter = 0
    clap_counter = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST]
            right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST]
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER]
            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER]

            # проверка поднятия рук
            hands_up = (left_wrist.y < left_shoulder.y) and (right_wrist.y < right_shoulder.y)
            if hands_stage == "down" and hands_up:
                hands_up_counter += 1
                hands_stage = "up"
            elif hands_stage == "up" and not hands_up:
                hands_stage = "down"

            # проверка хлопка (запястья близко друг к другу)
            hands_close = dist(left_wrist, right_wrist) < 0.1
            if clap_stage == "apart" and hands_close:
                clap_counter += 1
                clap_stage = "together"
            elif clap_stage == "together" and not hands_close:
                clap_stage = "apart"

            # рисуем скелет
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.putText(frame, f"Hands Up: {hands_up_counter}", (10, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Claps: {clap_counter}", (10, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        cv2.imshow("Exercise Counter", frame)

        if cv2.waitKey(1) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import mediapipe as mp
import math

mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
mp_style = mp.solutions.drawing_styles

mp_pose = mp.solutions.pose

def dist(p1, p2):
    return math.sqrt((p1.x - p2.x)**2 + (p1.y - p2.y)**2)

def smile_intensity(face):
    left_corner = face[61]
    right_corner = face[291]
    top_lip = face[13]
    bottom_lip = face[14]

    mouth_center_y = (top_lip.y + bottom_lip.y) / 2
    middle_point = ((left_corner.x + right_corner.x)/2, (left_corner.y + right_corner.y)/2)

    left_lift = mouth_center_y - left_corner.y
    right_lift = mouth_center_y - right_corner.y

    mouth_width = dist(left_corner, right_corner)
    smile_score = ((left_lift + right_lift)/2) / mouth_width
    smile_score = max(0.0, min(smile_score, 0.2))
    intensity = int(smile_score / 0.2 * 100)

    smiling = intensity > 10
    return smiling, middle_point, intensity

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cv2.namedWindow("Exercise + Face Detection", cv2.WINDOW_NORMAL)

hands_stage = "down"
clap_stage = "apart"
hands_up_counter = 0
clap_counter = 0

with mp_pose.Pose(min_detection_confidence=0.5,
                  min_tracking_confidence=0.5) as pose, \
     mp_face_mesh.FaceMesh(max_num_faces=1,
                           refine_landmarks=True,
                           min_detection_confidence=0.5,
                           min_tracking_confidence=0.5) as face_mesh:

    BODY_CONNECTIONS = [c for c in mp_pose.POSE_CONNECTIONS if c[0] > 10 and c[1] > 10]

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        pose_results = pose.process(rgb)
        if pose_results.pose_landmarks:
            landmarks = pose_results.pose_landmarks.landmark
            left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST]
            right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST]
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER]
            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER]

            # поднятие рук
            hands_up = (left_wrist.y < left_shoulder.y) and (right_wrist.y < right_shoulder.y)
            if hands_stage == "down" and hands_up:
                hands_up_counter += 1
                hands_stage = "up"
            elif hands_stage == "up" and not hands_up:
                hands_stage = "down"

            # хлопок
            hands_close = dist(left_wrist, right_wrist) < 0.1
            if clap_stage == "apart" and hands_close:
                clap_counter += 1
                clap_stage = "together"
            elif clap_stage == "together" and not hands_close:
                clap_stage = "apart"

            # линии тела
            mp_drawing.draw_landmarks(
                frame,
                pose_results.pose_landmarks,
                BODY_CONNECTIONS,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(0,200,0), thickness=2)
            )

            # точки тела
            for i, lm in enumerate(landmarks):
                if i > 10:
                    x = int(lm.x * frame.shape[1])
                    y = int(lm.y * frame.shape[0])
                    cv2.circle(frame, (x, y), 4, (0,255,0), -1)

        face_results = face_mesh.process(rgb)
        smile = False
        intensity = 0
        mouth_middle = (0,0)
        eyes_open = False

        if face_results.multi_face_landmarks:
            face = face_results.multi_face_landmarks[0].landmark

            smile, mouth_middle, intensity = smile_intensity(face)

            left_eye_EAR = dist(face[159], face[145])
            right_eye_EAR = dist(face[386], face[374])
            eyes_open = left_eye_EAR > 0.015 and right_eye_EAR > 0.015

            # маска лица
            mp_drawing.draw_landmarks(
                frame,
                face_results.multi_face_landmarks[0],
                connections=mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_style.get_default_face_mesh_tesselation_style()
            )

            # контур губ
            mp_drawing.draw_landmarks(
                frame,
                face_results.multi_face_landmarks[0],
                connections=mp_face_mesh.FACEMESH_LIPS,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_style.get_default_face_mesh_contours_style()
            )

            # контур глаз
            mp_drawing.draw_landmarks(
                frame,
                face_results.multi_face_landmarks[0],
                connections=mp_face_mesh.FACEMESH_LEFT_EYE,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_style.get_default_face_mesh_contours_style()
            )
            mp_drawing.draw_landmarks(
                frame,
                face_results.multi_face_landmarks[0],
                connections=mp_face_mesh.FACEMESH_RIGHT_EYE,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_style.get_default_face_mesh_contours_style()
            )

        cv2.putText(frame, f"Smile: {smile} ({intensity}%)", (10, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0) if smile else (0, 0, 255), 2)
        cv2.putText(frame, f"Eyes:  {eyes_open}", (10, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0) if eyes_open else (0, 0, 255), 2)
        cv2.putText(frame, f"Hands Up: {hands_up_counter}", (10, 120),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Claps: {clap_counter}", (10, 160),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # точка посередине рта
        cv2.circle(frame, (int(mouth_middle[0]*frame.shape[1]), int(mouth_middle[1]*frame.shape[0])), 5, (0,0,255), -1)

        cv2.imshow("Exercise + Face Detection", frame)

        if cv2.waitKey(1) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
smile_cascade = cv2.CascadeClassifier('haarcascade_smile.xml')

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Детекция лица
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
    eyes_open = False
    smiling = False

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]

        # Детекция глаз
        eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=10)
        if len(eyes) > 0:
            eyes_open = True
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)

        # Детекция улыбки
        smiles = smile_cascade.detectMultiScale(roi_gray, scaleFactor=1.7, minNeighbors=20)
        if len(smiles) > 0:
            smiling = True
        for (sx, sy, sw, sh) in smiles:
            cv2.rectangle(roi_color, (sx, sy), (sx + sw, sy + sh), (0, 0, 255), 2)

    cv2.putText(frame, f"Eyes: {eyes_open}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0) if eyes_open else (0, 0, 255), 2)
    cv2.putText(frame, f"Smile: {smiling}", (10, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0) if smiling else (0, 0, 255), 2)

    cv2.imshow("Face, Eyes & Smile Detection", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # Esc для выхода
        break

cap.release()
cv2.destroyAllWindows()
